In [5]:
import pandas as pd
import pickle
from datetime import datetime

### test

In [6]:
with open('data/invoices_new.pkl', 'rb') as f:
        test_invoices = pickle.load(f)

test_df = pd.DataFrame(test_invoices)
print(test_df)

         id  created_on                                              items
0   365371O  2019-05-27  [{'item': {'id': 142446, 'name': 'ii_142446', ...
1    389528  2019-03-19  [{'item': {'id': 172535, 'name': 'ii_172535', ...
2    341762  2019-06-19  [{'item': {'id': 143176, 'name': 'ii_143176', ...
3    319405  2019-03-27  [{'item': {'id': 141009, 'name': 'ii_141009', ...
4    343695  2019-04-10  [{'item': {'id': 144025, 'name': 'ii_144025', ...
..      ...         ...                                                ...
95   356552  2019-03-21  [{'item': {'id': 112495, 'name': 'ii_112495', ...
96   306163  2019-09-18  [{'item': {'id': 136017, 'name': 'ii_136017', ...
97  373375O  2019-08-24  [{'item': {'id': 180383, 'name': 'ii_180383', ...
98   321243  2019-05-26  [{'item': {'id': 124775, 'name': 'ii_124775', ...
99   377960  2019-02-22  [{'item': {'id': 123242, 'name': 'ii_123242', ...

[100 rows x 3 columns]


In [7]:
test_df['items'].iloc[2]

[{'item': {'id': 143176, 'name': 'ii_143176', 'unit_price': 175, 'type': 2},
  'quantity': 2},
 {'item': {'id': 161765, 'name': 'ii_161765', 'unit_price': 170, 'type': 2},
  'quantity': 8},
 {'item': {'id': 167763, 'name': 'ii_167763', 'unit_price': 150, 'type': 3},
  'quantity': 5},
 {'item': {'id': 195203, 'name': 'ii_195203', 'unit_price': 194, 'type': 0},
  'quantity': 3},
 {'item': {'id': 188321, 'name': 'ii_188321', 'unit_price': 101, 'type': 3},
  'quantity': 1},
 {'item': {'id': 114990, 'name': 'ii_114990', 'unit_price': 132, 'type': 3},
  'quantity': 2},
 {'item': {'id': 126940, 'name': 'ii_126940', 'unit_price': 166, 'type': 0},
  'quantity': 3}]

# DataExtractor class

In [12]:
class DataExtractor:

    def __init__(self, invoice_file, expired_ids_file):
        self.invoice_file = invoice_file
        self.expired_ids_file = expired_ids_file
        self.invoices = None
        self.expired_ids = None


    def load_data(self):

        with open(self.invoice_file, 'rb') as f:
            self.invoices = pickle.load(f)

        

        with open(self.expired_ids_file, 'r') as f:
            ids = f.read()
            self.expired_ids = list(ids.split(', '))

    def transform_data(self):

        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}

        transformed_data  = []


        # logic:
        # for each invoice in invoices, set the invoice_id, created_on, is_expired (true if its contained in expired_ids)
        # each row in 'items' is a list of dictionaries, each dictionary has the keys 'item' and 'quantity'
        # the value for each key 'item' is a dictionary itself containing other keys 
        # for total_invoice_price we need to multiply 'unit_price" by 'quantity'
        # and for invoiceitem_id and invoiceitem_name a list
        #
        # code is unfortanately not complete and has errors regarding assigning the items

        
        for invoice in self.invoices:
    
            invoice_id = invoice['id']
            created_on = pd.to_datetime(invoice['created_on'])
            is_expired = invoice_id in self.expired_ids
    
           
    
    
            for item in invoice['items']:
                 
                invoice_id, invoiceitem_id, invoiceitem_name, type, unit_price = [],[],[],[],[]
                total_invoice_price = 0
                for item2 in item:
                    invoiceitem_id.append(item2['id'])
                    invoiceitem_id.append(item2['name'])
                    type.append(type_conversion[item2['type']])
                    unit_price.append(item2['unit_price'])
                    total_price = item2['unit_price'] * item['quantity']

                    total_invoice_price =+ total_price
                                
                percentage_in_invoice = total_price / total_invoice_price
    
    
                transformed_data.append([
                    invoice_id,
                    invoiceitem_id,
                    invoiceitem_name,
                    created_on,
                    is_expired,
                    type,
                    unit_price,
                    total_price,
                    percentage_in_invoice
                    
                ])

        
    
        df = pd.DataFrame(transfromed_data, columns = columns)
    
        df['invoice_id'] = df['invoice_id'].astype(int)
        
        # chatgpt citation:
        # prompt: ensure the columns of the dataframe df are as the same type mentioned below similiar to this code: df['invoice_id'] = df['invoice_id'].astype(int)
        #         invoice_id: int
        #         created_on: datetime64[ns]
        #         invoiceitem_id: int
        #         invoiceitem_name: str
        #         type: str (use this conversion table: {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'})
        #         unit_price: int
        #         total_price: int (unit_price*quantity)
        #         percentage_in_invoice: float (unit_price*quantity / invoice_total)
        #         is_expired: bool (whether invoice_id is contained expired_invoices.txt or not)
    
    
        df['invoice_id'] = df['invoice_id'].astype(int)
        df['invoiceitem_id'] = df['invoiceitem_id'].astype(int)
        df['invoiceitem_name'] = df['invoiceitem_name'].astype(str)
        df['created_on'] = pd.to_datetime(df['created_on'])
        df['is_expired'] = df['is_expired'].astype(bool)
        df['type'] = df['type'].astype(str)
        df['unit_price'] = df['unit_price'].astype(int)
        df['total_price'] = df['total_price'].astype(int)
        df['percentage_in_invoice'] = df['percentage_in_invoice'].astype(float)

        df = df.sort_values(by=['invoice_id', 'invoiceitem_id']).reset_index(drop=True)

        return df


    

In [13]:
invoice_file = 'data/invoices_new.pkl'
expired_ids_file = 'data/expired_invoices.txt'

data_extractor = DataExtractor(invoice_file, expired_ids_file)
data_extractor.load_data()

transformed_df = data_extractor.transform_data()

TypeError: string indices must be integers, not 'str'

In [65]:
transformed_df

NameError: name 'transformed_df' is not defined